# GDELT Dataset Generation

This notebook demonstrates how to use GDELT (Global Database of Events, Language, and Tone) as a data source for generating forecasting questions. GDELT provides access to a massive database of global news articles via BigQuery, offering broader coverage than standard news search.

In [1]:
%pip install -e ..

from IPython.display import clear_output
clear_output()

In [ ]:
import os
from lightningrod import LightningRod

api_key = os.getenv("LIGHTNINGROD_API_KEY")
if not api_key:
    raise ValueError("LIGHTNINGROD_API_KEY is not set")

client = LightningRod(api_key=api_key)

## Configure GDELT Seed Generator

GDELT provides access to a much larger corpus of news articles than standard Google News search. It's particularly useful for:
- Global events and international news
- Historical analysis
- Large-scale dataset generation

The `GdeltSeedGenerator` queries BigQuery to fetch articles, allowing you to process thousands of articles per interval.

In [3]:
from datetime import datetime
from lightningrod import GdeltSeedGenerator

gdelt_seed_generator = GdeltSeedGenerator(
    start_date=datetime(2025, 1, 1),
    end_date=datetime(2025, 1, 31),
    interval_step_days=7,
    articles_per_interval=1000,
)

## When to Use GDELT vs Google News

**Use GDELT when:**
- You need access to a very large number of articles
- You're analyzing global or international events
- You need historical data
- You want broader coverage across many sources

**Use Google News when:**
- You need recent, curated news articles
- You want more control over search queries
- You're working with smaller, focused datasets
- You need faster iteration on specific topics

In [4]:
from lightningrod import AnswerType, AnswerTypeEnum, QuestionGenerator, FilterCriteria

answer_type = AnswerType(answer_type=AnswerTypeEnum.BINARY)

question_generator = QuestionGenerator(
    instructions=(
        "Generate forward-looking questions about global events and international news. "
        "Questions should focus on future outcomes that can be verified."
    ),
    examples=[
        "Will the conflict in region X escalate in the next month?",
        "Will country Y sign the trade agreement this quarter?",
        "Will the international summit achieve its stated goals?",
    ],
    bad_examples=[
        "What happened in the conflict?",
        "When was the trade agreement signed?",
        "Who attended the summit?",
    ],
    filter_=FilterCriteria(
        rubric="The question should be forward-looking and about future global events",
        min_score=0.7
    ),
    answer_type=answer_type,
)

In [5]:
from lightningrod import WebSearchLabeler, QuestionRenderer

labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

renderer = QuestionRenderer(
    answer_type=answer_type,
)

## Run the Pipeline

The pipeline works the same way as with Google News - GDELT is just a different data source.

In [ ]:
from lightningrod import QuestionPipeline

pipeline_config = QuestionPipeline(
    seed_generator=gdelt_seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

dataset = client.transforms.run(pipeline_config, max_questions=10) # keep max questions low when testing

In [7]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [8]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
print(df.head())

Generated 1 samples

                              question.question_text label.label  \
0  Will Donald Trump be sentenced in the hush mon...           1   

   label.label_confidence label.resolution_date  \
0                     1.0   2025-01-10T00:00:00   

                                     label.reasoning  \
0  Donald Trump was sentenced in the hush money c...   

                                label.answer_sources  \
0  https://vertexaisearch.cloud.google.com/ground...   

                                              prompt  \
0  QUESTION:\nWill Donald Trump be sentenced in t...   

                                      seed.seed_text  \
0  Title: ABC News – Breaking News, Latest News a...   

                                            seed.url seed.seed_creation_date  \
0  https://abcnews.go.com/Politics/wireStory/trum...     2025-01-10T00:00:00   

   is_valid                        meta.sample_id  \
0      True  548dd97c-b733-4f8c-9f72-c754e7be4d35   

                  m